# Prerequisites: Getting Started with the Relational AI Native App
Before you can begin, you need to install the Relational AI Native App in your Snowflake account. Here’s how to get access:

## 1. [Find the App](https://app.snowflake.com/marketplace/listing/GZTYZOOIX8H/relationalai-relationalai?search=Relational%20AI) on the Snowflake Marketplace

The Relational AI Native App is available directly from the Snowflake Marketplace. You can request to have your account authorized for installation from the app's listing page.

## 2. Request a Trial Version

To get started with a trial, please contact your Snowflake Account Team. They will connect you with the Relational AI team to get your trial set up.


# Client Wealth Map
## 360-Degree View & Understanding of Asset Management Clients. 
## The goal is to map every aspect of our client's life. 
## This will allow us to gain deeper insights of our clients, allowing us to create personalized services that delight them.  
### 1. Client's current asset ownership.
### 2. Client's goals & preferences.
### 3. Client's risk preference. 
### 4. Client's family situation.  

### Author: [Prasanna Rajagopal](https://www.linkedin.com/in/prasannarajagopal/)

#### Date: June 2025

# Two Broad Steps to Create a Client Graph Network
- ### Using Unstructured Documents Using Content From Emails, Trust Documents.
- ### Using Structured Content About Client Already in a Table.
## Step 1: Extract the Graph Nodes from Emails, Trust Documents, etc.
- ### Using Snowflake Cortex ```AI_COMPLETE``` Structured Outputs. 
- ### Stored the Extracted Nodes in a Table.  
## Step 2: Create the Graph Network with Nodes and Edges Using the Extracted Information.
- ### Using ```Relational AI``` Graph Co-Processor on Snowflake.  

## Extract Subject, Predicate, and Object From Emails.   

In [ ]:
SELECT * FROM CLIENT_EMAILS;

In [ ]:
SELECT
    CLIENT_NAME, AI_COMPLETE(
        model => 'claude-3-5-sonnet', -- You can use the model of your choice
        prompt => 'From the following email content, please extract the client''s key financial goals. For each goal, provide a single-word description for the goal (e.g., HOME, INCOME, CAR), the type of action (e.g., OWN, LIVING EXPENSE, BUY), and the budget as a number without commas or currency symbols. Match the output to the requested JSON schema precisely. Email Content: ' || EMAIL_BODY,

        -- The JSON schema is defined here to structure the AI model's response.
        -- It has been updated to categorize goals and extract numeric values.
        response_format => PARSE_JSON('{
            "type": "json",
            "schema": {
                "type": "object",
                "properties": {
                    "goals": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "GOAL_DESCRIPTION": {
                                    "type": "string"
                                },
                                "GOAL_TYPE": {
                                    "type": "string"
                                },
                                "GOAL_BUDGET": {
                                    "type": "string"
                                }
                            },
                            "required": ["GOAL_DESCRIPTION", "GOAL_TYPE"]
                        }
                    }
                }
            }
        }')
    ) AS CLIENT_FINANCIAL_SUMMARY
FROM
    CLIENT_EMAILS;

## Prerequisites:
- ### Create a Database & Schema called RAI_DEMO.SIMPLE_START.
- ### Ensure you have installed the Relational AI (RAI) App on your account from the Snowflake Marketplace.  
    - ### You may need to reach out the Relational AI team via the Listing in the Snowflake Marketplace to discuss getting a trial license.  

## Once the service is installed and started, you can use the following commands to check on the status of the service and activate or deactivate the service.   

## Check the Status of Relational AI Service

In [ ]:
CALL RELATIONALAI.APP.SERVICE_STATUS();

## Activate Relational AI Service

In [ ]:
session = get_active_session()
session.sql("CALL RELATIONALAI.APP.ACTIVATE();").collect()

## Deactivate Relational AI, when not in use, to save money :dollar: :moneybag:

In [ ]:
session = get_active_session()
session.sql("CALL RELATIONALAI.APP.DEACTIVATE();").collect()

# What is Relational AI? 
- ### Relational AI is a system that combines relational knowledge graphs with data clouds to enhance AI capabilities, specifically within platforms like Snowflake. 
- ### It acts as an "AI coprocessor" for data clouds, enabling businesses to analyze interconnected data, apply rules-based reasoning, and perform graph analytics for intelligent applications.
## Relational AI is available as a Snowflake Native App in the Snowflake Marketplace.  
# What is a Snowflake Native App?
- ### The Snowflake Native App Framework is a platform that allows providers to build, distribute, and sell data-intensive applications directly within the Snowflake Data Cloud. 
- ### These applications, called Snowflake Native Apps, run entirely within the consumer's Snowflake account, eliminating the need for data movement or copying. 
- ### This framework offers a streamlined development experience, enhanced security, and the ability to leverage Snowflake's built-in features. 
# Snowflake + Relational AI Demo
- ## Easy Ingestion of Data From Snowflake
    - ### Don't have to move the Data
- ## Easy Graph Build Out. 
- ## Query one specific Client. 
- ## Draw the graph out.  
- ## Graph Reasoning
    - ### Jaccard Similarity. 
# What's Next? 
- ## You can do a lot more with Snowflake & Relational AI. Here are some possibilities:  
    - ### Unmasking influence in your graph network using Centrality Measures.  
        - ### Gain a deeper understanding of your client's needs & wants.
    - ### Community Detection
        - ### Hyper-Personalization and Client Segmentation
            - Community detection can automatically segment clients into highly nuanced groups based on their holistic financial DNA, not just simple demographics.
    - ### Enhanced Product and Service Development
        - ### Community detection might uncover an emerging community of clients who are all investing in sustainable and ESG (Environmental, Social, and Governance) assets, linked to portfolios with "Ethical Investing" goals. 
        - ### This community might be geographically dispersed and demographically diverse, making it hard to spot with traditional analysis.
        - ### The firm can use this insight to justify developing a new suite of ESG-focused investment products, knowing there is a well-defined, organic client base for it. 
        - ### They can also create educational content and advisory services specifically for this community.
            


        

## Install the Relational AI Python Module in your Notebook Environment.  

In [ ]:
%pip install relationalai

## Python Import Statements.  

In [1]:
import random
import colorsys

import relationalai as rai
from relationalai.std import aggregates, rel, alias
from relationalai.std.graphs import Graph
import networkx as nx
import matplotlib.pyplot as plt
random.seed(123)

provider = rai.Provider()

## Create Table in Snowflake to store the sample client and portfolio data.  

In [ ]:
CREATE OR REPLACE TABLE RAI_DEMO.SIMPLE_START.AssetOwnership (
    -- ID INT AUTO_INCREMENT PRIMARY KEY,
    Client_ID VARCHAR(50) NOT NULL,
    Asset_Type VARCHAR(20) NOT NULL,
    Asset_Type_Specific VARCHAR(10) NOT NULL,
    Quantity INT NOT NULL,
    Purchase_Price DECIMAL(10, 2) NOT NULL,
    Current_Price DECIMAL(10, 2) NOT NULL,
    Purchase_Date DATE NOT NULL
);

## Only execute the following SQL if you wish to alter the table in any way.  

In [ ]:
ALTER TABLE RAI_DEMO.SIMPLE_START.AssetOwnership
ALTER COLUMN Asset_Type VARCHAR(20);

## Insert Rows into Asset Ownership Tables
### Execute these insert statements after you create the table in Snowflake.  

In [ ]:
INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_A', 'EQUITY', 'AAPL', 150, 150.25, 175.80, '2023-01-15'),
('CLIENT_A', 'ETF', 'FLIN', 500, 25.10, 27.50, '2023-03-20'),
('CLIENT_A', 'EQUITY', 'NVDA', 50, 450.00, 520.10, '2023-07-01'),
('CLIENT_A', 'ETF', 'DIVI', 300, 40.50, 41.20, '2023-09-10'),
('CLIENT_A', 'EQUITY', 'XOM', 200, 95.80, 98.00, '2023-11-05');

INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_B', 'ETF', 'FLJP', 400, 30.20, 31.80, '2023-02-01'),
('CLIENT_B', 'EQUITY', 'PFE', 250, 42.10, 41.50, '2023-04-10'),
('CLIENT_B', 'ETF', 'FLQM', 600, 22.00, 23.50, '2023-06-15'),
('CLIENT_B', 'EQUITY', 'ADBE', 75, 550.00, 580.30, '2023-08-22'),
('CLIENT_B', 'ETF', 'FLQL', 350, 35.70, 36.90, '2023-10-28');

INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_C', 'EQUITY', 'CVX', 180, 160.00, 162.50, '2023-03-01'),
('CLIENT_C', 'ETF', 'DIVI', 550, 41.00, 40.80, '2023-05-12'),
('CLIENT_C', 'EQUITY', 'ADBE', 80, 560.00, 590.20, '2023-07-25'),
('CLIENT_C', 'ETF', 'FLIN', 450, 26.00, 28.10, '2023-09-18'),
('CLIENT_C', 'EQUITY', 'PFE', 300, 43.00, 42.80, '2023-12-01');

INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_D', 'ETF', 'FLQM', 700, 21.50, 22.90, '2023-01-25'),
('CLIENT_D', 'EQUITY', 'XOM', 220, 96.50, 99.10, '2023-04-05'),
('CLIENT_D', 'ETF', 'FLJP', 380, 31.00, 32.50, '2023-06-20'),
('CLIENT_D', 'EQUITY', 'AAPL', 120, 155.00, 180.50, '2023-08-08'),
('CLIENT_D', 'ETF', 'FLQL', 420, 36.00, 37.20, '2023-10-15');

INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_E', 'EQUITY', 'NVDA', 60, 460.00, 530.40, '2023-02-14'),
('CLIENT_E', 'ETF', 'DIVI', 480, 40.70, 41.50, '2023-05-01'),
('CLIENT_E', 'EQUITY', 'CVX', 190, 162.00, 165.30, '2023-07-10'),
('CLIENT_E', 'ETF', 'FLIN', 520, 25.50, 27.90, '2023-09-05'),
('CLIENT_E', 'EQUITY', 'PFE', 280, 42.50, 42.10, '2023-11-20');

## Create more sample data with client's portfolios.  

In [ ]:
INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_F', 'EQUITY', 'MSFT', 120, 280.50, 310.20, '2023-01-05'),
('CLIENT_F', 'ETF', 'FLQM', 300, 23.00, 24.50, '2023-02-18'),
('CLIENT_F', 'MUTUAL FUND', 'VFINX', 250, 120.00, 125.70, '2023-03-22'),
('CLIENT_F', 'EQUITY', 'GOOG', 70, 100.00, 115.80, '2023-04-01'),
('CLIENT_F', 'ETF', 'DIVI', 400, 42.00, 43.10, '2023-05-10'),
('CLIENT_F', 'MUTUAL FUND', 'SWPPX', 180, 80.00, 83.20, '2023-06-15'),
('CLIENT_F', 'EQUITY', 'AMZN', 40, 130.00, 145.60, '2023-07-20'),
('CLIENT_F', 'ETF', 'FLJP', 500, 31.00, 32.50, '2023-08-25'),
('CLIENT_F', 'MUTUAL FUND', 'FNILX', 320, 55.00, 57.80, '2023-09-01'),
('CLIENT_F', 'EQUITY', 'TSLA', 30, 220.00, 240.10, '2023-10-08'),
('CLIENT_F', 'ETF', 'FLIN', 280, 26.00, 28.00, '2023-11-12'),
('CLIENT_F', 'MUTUAL FUND', 'PRWCX', 200, 90.00, 93.50, '2023-12-01'),
('CLIENT_F', 'EQUITY', 'JPM', 150, 140.00, 148.00, '2024-01-03'),
('CLIENT_F', 'ETF', 'FLQL', 350, 37.00, 38.50, '2024-02-10'),
('CLIENT_F', 'MUTUAL FUND', 'VIIIX', 100, 70.00, 72.30, '2024-03-05');

INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_G', 'EQUITY', 'V', 90, 240.00, 255.60, '2023-01-10'),
('CLIENT_G', 'ETF', 'FLIN', 450, 25.50, 27.80, '2023-02-20'),
('CLIENT_G', 'MUTUAL FUND', 'SWPPX', 220, 82.00, 85.10, '2023-03-25'),
('CLIENT_G', 'EQUITY', 'PG', 80, 150.00, 158.30, '2023-04-05'),
('CLIENT_G', 'ETF', 'FLJP', 380, 30.50, 31.90, '2023-05-15'),
('CLIENT_G', 'MUTUAL FUND', 'VIIIX', 150, 71.00, 73.50, '2023-06-20'),
('CLIENT_G', 'EQUITY', 'KO', 200, 60.00, 62.50, '2023-07-25'),
('CLIENT_G', 'ETF', 'FLQM', 550, 22.50, 24.00, '2023-08-30'),
('CLIENT_G', 'MUTUAL FUND', 'VFINX', 300, 121.00, 126.50, '2023-09-05'),
('CLIENT_G', 'EQUITY', 'DIS', 100, 105.00, 110.80, '2023-10-10'),
('CLIENT_G', 'ETF', 'DIVI', 320, 41.50, 42.60, '2023-11-15'),
('CLIENT_G', 'MUTUAL FUND', 'FNILX', 280, 56.00, 58.50, '2023-12-05'),
('CLIENT_G', 'EQUITY', 'NFLX', 50, 400.00, 425.70, '2024-01-08'),
('CLIENT_G', 'ETF', 'FLQL', 400, 36.50, 37.90, '2024-02-15'),
('CLIENT_G', 'MUTUAL FUND', 'PRWCX', 180, 91.00, 94.20, '2024-03-10');

INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_H', 'EQUITY', 'HD', 80, 330.00, 345.50, '2023-01-15'),
('CLIENT_H', 'ETF', 'FLQL', 300, 37.00, 38.60, '2023-02-25'),
('CLIENT_H', 'MUTUAL FUND', 'VFINX', 200, 122.00, 127.80, '2023-03-30'),
('CLIENT_H', 'EQUITY', 'UNH', 60, 500.00, 520.00, '2023-04-10'),
('CLIENT_H', 'ETF', 'FLIN', 500, 26.50, 28.20, '2023-05-20'),
('CLIENT_H', 'MUTUAL FUND', 'SWPPX', 250, 83.00, 86.00, '2023-06-25'),
('CLIENT_H', 'EQUITY', 'MSFT', 110, 285.00, 315.50, '2023-07-30'),
('CLIENT_H', 'ETF', 'DIVI', 420, 42.00, 43.30, '2023-09-01'),
('CLIENT_H', 'MUTUAL FUND', 'FNILX', 350, 57.00, 59.50, '2023-10-05'),
('CLIENT_H', 'EQUITY', 'GOOG', 65, 102.00, 118.00, '2023-11-10'),
('CLIENT_H', 'ETF', 'FLJP', 390, 31.50, 33.00, '2023-12-15'),
('CLIENT_H', 'MUTUAL FUND', 'VIIIX', 120, 72.00, 74.80, '2024-01-20');

INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_I', 'EQUITY', 'AMZN', 35, 135.00, 150.20, '2023-01-20'),
('CLIENT_I', 'ETF', 'FLQM', 600, 21.80, 23.30, '2023-03-01'),
('CLIENT_I', 'MUTUAL FUND', 'PRWCX', 280, 92.00, 95.50, '2023-04-05'),
('CLIENT_I', 'EQUITY', 'TSLA', 25, 230.00, 250.80, '2023-05-10'),
('CLIENT_I', 'ETF', 'FLIN', 480, 27.00, 29.00, '2023-06-15'),
('CLIENT_I', 'MUTUAL FUND', 'VFINX', 260, 123.00, 128.90, '2023-07-20'),
('CLIENT_I', 'EQUITY', 'JPM', 140, 142.00, 150.00, '2023-08-25'),
('CLIENT_I', 'ETF', 'FLQL', 360, 37.50, 39.00, '2023-09-30'),
('CLIENT_I', 'MUTUAL FUND', 'SWPPX', 190, 84.00, 87.20, '2023-11-05'),
('CLIENT_I', 'EQUITY', 'V', 85, 245.00, 260.80, '2023-12-10'),
('CLIENT_I', 'ETF', 'DIVI', 380, 42.50, 43.80, '2024-01-15');

INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_J', 'EQUITY', 'PG', 75, 152.00, 160.50, '2023-02-01'),
('CLIENT_J', 'ETF', 'FLJP', 420, 30.80, 32.30, '2023-03-10'),
('CLIENT_J', 'MUTUAL FUND', 'VIIIX', 160, 73.00, 75.80, '2023-04-15'),
('CLIENT_J', 'EQUITY', 'KO', 180, 61.00, 63.50, '2023-05-20'),
('CLIENT_J', 'ETF', 'FLQM', 580, 22.00, 23.50, '2023-06-25'),
('CLIENT_J', 'MUTUAL FUND', 'FNILX', 310, 58.00, 60.50, '2023-07-30'),
('CLIENT_J', 'EQUITY', 'DIS', 95, 108.00, 113.80, '2023-09-05'),
('CLIENT_J', 'ETF', 'FLIN', 460, 27.50, 29.50, '2023-10-10'),
('CLIENT_J', 'MUTUAL FUND', 'PRWCX', 210, 93.00, 96.80, '2023-11-15'),
('CLIENT_J', 'EQUITY', 'NFLX', 45, 410.00, 435.90, '2023-12-20'),
('CLIENT_J', 'ETF', 'FLQL', 330, 38.00, 39.50, '2024-01-25'),
('CLIENT_J', 'MUTUAL FUND', 'VFINX', 290, 124.00, 129.90, '2024-02-28');

INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_K', 'EQUITY', 'HD', 70, 335.00, 350.50, '2023-03-05'),
('CLIENT_K', 'ETF', 'DIVI', 400, 43.00, 44.50, '2023-04-10'),
('CLIENT_K', 'MUTUAL FUND', 'SWPPX', 230, 85.00, 88.00, '2023-05-15'),
('CLIENT_K', 'EQUITY', 'UNH', 55, 510.00, 530.00, '2023-06-20'),
('CLIENT_K', 'ETF', 'FLIN', 470, 28.00, 30.00, '2023-07-25'),
('CLIENT_K', 'MUTUAL FUND', 'FNILX', 300, 59.00, 61.50, '2023-09-01'),
('CLIENT_K', 'EQUITY', 'MSFT', 100, 290.00, 320.50, '2023-10-05'),
('CLIENT_K', 'ETF', 'FLJP', 410, 32.00, 33.50, '2023-11-10'),
('CLIENT_K', 'MUTUAL FUND', 'PRWCX', 240, 94.00, 97.80, '2023-12-15'),
('CLIENT_K', 'EQUITY', 'GOOG', 60, 105.00, 120.00, '2024-01-20'),
('CLIENT_K', 'ETF', 'FLQM', 500, 22.50, 24.00, '2024-02-25');

INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_L', 'EQUITY', 'AMZN', 30, 140.00, 155.50, '2023-03-10'),
('CLIENT_L', 'ETF', 'FLQL', 340, 38.50, 40.00, '2023-04-15'),
('CLIENT_L', 'MUTUAL FUND', 'VFINX', 270, 125.00, 130.80, '2023-05-20'),
('CLIENT_L', 'EQUITY', 'TSLA', 20, 240.00, 260.10, '2023-06-25'),
('CLIENT_L', 'ETF', 'DIVI', 350, 43.50, 45.00, '2023-07-30'),
('CLIENT_L', 'MUTUAL FUND', 'VIIIX', 130, 74.00, 76.50, '2023-09-05'),
('CLIENT_L', 'EQUITY', 'JPM', 130, 145.00, 153.00, '2023-10-10'),
('CLIENT_L', 'ETF', 'FLIN', 490, 28.50, 30.50, '2023-11-15'),
('CLIENT_L', 'MUTUAL FUND', 'SWPPX', 260, 86.00, 89.20, '2023-12-20'),
('CLIENT_L', 'EQUITY', 'V', 80, 250.00, 265.80, '2024-01-25'),
('CLIENT_L', 'ETF', 'FLJP', 370, 32.50, 34.00, '2024-03-01'),
('CLIENT_L', 'MUTUAL FUND', 'PRWCX', 220, 95.00, 98.80, '2024-04-05');

INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_M', 'EQUITY', 'PG', 70, 155.00, 163.50, '2023-03-15'),
('CLIENT_M', 'ETF', 'FLQM', 520, 22.80, 24.30, '2023-04-20'),
('CLIENT_M', 'MUTUAL FUND', 'FNILX', 330, 60.00, 62.50, '2023-05-25'),
('CLIENT_M', 'EQUITY', 'KO', 170, 62.00, 64.50, '2023-06-30'),
('CLIENT_M', 'ETF', 'FLQL', 350, 39.00, 40.50, '2023-08-05'),
('CLIENT_M', 'MUTUAL FUND', 'VFINX', 310, 126.00, 131.90, '2023-09-10'),
('CLIENT_M', 'EQUITY', 'DIS', 90, 110.00, 115.80, '2023-10-15'),
('CLIENT_M', 'ETF', 'DIVI', 360, 44.00, 45.50, '2023-11-20'),
('CLIENT_M', 'MUTUAL FUND', 'VIIIX', 140, 75.00, 77.50, '2023-12-25'),
('CLIENT_M', 'EQUITY', 'NFLX', 40, 420.00, 445.90, '2024-01-30');

INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_N', 'EQUITY', 'HD', 65, 340.00, 355.50, '2023-04-01'),
('CLIENT_N', 'ETF', 'FLIN', 440, 29.00, 31.00, '2023-05-05'),
('CLIENT_N', 'MUTUAL FUND', 'PRWCX', 250, 96.00, 99.80, '2023-06-10'),
('CLIENT_N', 'EQUITY', 'UNH', 50, 520.00, 540.00, '2023-07-15'),
('CLIENT_N', 'ETF', 'FLJP', 400, 33.00, 34.50, '2023-08-20'),
('CLIENT_N', 'MUTUAL FUND', 'SWPPX', 240, 87.00, 90.20, '2023-09-25'),
('CLIENT_N', 'EQUITY', 'MSFT', 95, 295.00, 325.50, '2023-10-30'),
('CLIENT_N', 'ETF', 'FLQM', 530, 23.00, 24.50, '2023-12-05'),
('CLIENT_N', 'MUTUAL FUND', 'FNILX', 280, 61.00, 63.50, '2024-01-10'),
('CLIENT_N', 'EQUITY', 'GOOG', 55, 108.00, 123.00, '2024-02-15');

INSERT INTO RAI_DEMO.SIMPLE_START.AssetOwnership (Client_ID, Asset_Type, Asset_Type_Specific, Quantity, Purchase_Price, Current_Price, Purchase_Date) VALUES
('CLIENT_O', 'EQUITY', 'AMZN', 28, 145.00, 160.50, '2023-04-05'),
('CLIENT_O', 'ETF', 'FLQL', 360, 39.50, 41.00, '2023-05-10'),
('CLIENT_O', 'MUTUAL FUND', 'VFINX', 280, 127.00, 132.80, '2023-06-15'),
('CLIENT_O', 'EQUITY', 'TSLA', 18, 250.00, 270.10, '2023-07-20'),
('CLIENT_O', 'ETF', 'DIVI', 380, 44.50, 46.00, '2023-08-25'),
('CLIENT_O', 'MUTUAL FUND', 'VIIIX', 150, 76.00, 78.50, '2023-09-30'),
('CLIENT_O', 'EQUITY', 'JPM', 125, 148.00, 156.00, '2023-10-05'),
('CLIENT_O', 'ETF', 'FLIN', 500, 29.50, 31.50, '2023-11-10'),
('CLIENT_O', 'MUTUAL FUND', 'SWPPX', 270, 88.00, 91.20, '2023-12-15'),
('CLIENT_O', 'EQUITY', 'V', 75, 255.00, 270.80, '2024-01-20'),
('CLIENT_O', 'ETF', 'FLJP', 390, 33.50, 35.00, '2024-02-25'),
('CLIENT_O', 'MUTUAL FUND', 'PRWCX', 230, 97.00, 100.80, '2024-03-30');


## Select From AssetOwnership

In [ ]:
SELECT COUNT(*) FROM RAI_DEMO.SIMPLE_START.AssetOwnership;

### Define Model in RelationalAI
Let's define our model object. **Models** represent collections of objects. **Objects**, like Python objects, have **types** and **properties**, which we will define in a bit.

In [ ]:
model = rai.Model("client_asset_ownership", ensure_change_tracking=True)

Asset_Ownership = model.Type("Asset_Ownership", source="rai_demo.SIMPLE_START.AssetOwnership")

In [ ]:
Asset_Ownership.known_properties()

In [ ]:
# count number of client ownership records available.  
with model.query() as select:
    t = Asset_Ownership()
    response = select(alias(aggregates.count(t), 'nr.assets_owned_by_clients'))
response

## Create Graph of Assets Owned by the Client. 
### This graph does not include the client id for the specific asset owned by the client.  

In [ ]:
# Assume 'model' is an existing context manager from the RelationalAI SDK.
# Assume 'Asset_Ownership' is a pre-existing base relation with columns:
# client_id, asset_type

# --- 1. Define Node Types ---
# A type for each unique client.
Client = model.Type("Client")

# A type for each unique asset type (e.g., "ETF", "Equity", "Bond").
AssetType = model.Type("AssetType")

# The specific Assets Owned by the client - XOM, AAPL, etc.
AssetTypeSpecific = model.Type("AssetTypeSpecific")


# --- 2. Populate Node Types from Source Data (with Cleaning) ---
# Import 'graphs' and the 'strings' module for data cleaning.
from relationalai.std import graphs, strings

# For each row in Asset_Ownership, ensure a Client entity exists for the given client_id.
# We use strings.strip and strings.lowercase to standardize the data, removing whitespace
# and making it case-insensitive to prevent duplicate nodes.
with model.rule():
    t = Asset_Ownership()
    cleaned_id = strings.lowercase(strings.strip(t.client_id))
    t.set(client_id = Client.add(client_id=cleaned_id))

# For each row in Asset_Ownership, ensure an AssetType entity exists for the given asset_type.
with model.rule():
    t = Asset_Ownership()
    cleaned_asset = strings.lowercase(strings.strip(t.asset_type))
    t.set(asset_type = AssetType.add(asset_type=cleaned_asset))

with model.rule():
    t = Asset_Ownership()
    cleaned_asset_specific = strings.lowercase(strings.strip(t.asset_type_specific))
    t.set(asset_type_specific = AssetTypeSpecific.add(asset_type_specific=cleaned_asset_specific,
                                                     client_id=t.client_id))
    # t.set(asset_type_specific = AssetTypeSpecific.add(client_id=t.client_id))


# --- 3. Define and Build the Graph ---
# Create a graph from the model. This will be our single source of truth.
clientAssetOwnershipGraph = graphs.Graph(model, "client_asset_ownership_graph")


# --- 4. Add Nodes to the Graph ---
# Add every entity in the 'Client' type as a node in the graph.
with model.rule():
    client = Client()
    client.set(clientAssetOwnershipGraph.Node)

# Add every entity in the 'AssetType' type as a node in the graph.
with model.rule():
    assetType = AssetType()
    assetType.set(clientAssetOwnershipGraph.Node)

with model.rule():
    assetTypeSpecific = AssetTypeSpecific()
    assetTypeSpecific.set(clientAssetOwnershipGraph.Node)

# --- 5. Add Edges to the Graph (with Cleaning) ---
# This is the key rule that connects the nodes correctly.
# For each transaction in Asset_Ownership, create an edge from the
# corresponding Client node to the corresponding AssetType node.
with model.rule():
    ownership = Asset_Ownership()
    
    # Define the two nodes that will form the edge for this specific ownership record.
    # We must strip and lowercase the values here as well to find the correct, cleaned nodes.
    from_node = Client(client_id = strings.lowercase(strings.strip(ownership.client_id)))
    to_node = AssetType(asset_type = strings.lowercase(strings.strip(ownership.asset_type)))

    # Add the edge connecting these two specific nodes.
    clientAssetOwnershipGraph.Edge.add(from_=from_node, to=to_node)
    
    from_node_asset_type_specific = AssetType(asset_type = strings.lowercase(strings.strip(ownership.asset_type)))
    to_node_asset_type_specific = AssetTypeSpecific(asset_type_specific = strings.lowercase(strings.strip(ownership.asset_type_specific)))

    # Add the edge connecting these two specific nodes.
    clientAssetOwnershipGraph.Edge.add(from_=from_node_asset_type_specific
                                            , to=to_node_asset_type_specific)
# --- 6. Configure Visualization ---
# Get the Node type from the graph we already created.
Node = clientAssetOwnershipGraph.Node

# Configure how Client nodes should look in the visualization.
# The label and hover text will be the client's ID.
Node.extend(Client, label=Client.client_id, hover=Client.client_id)

# Configure how AssetType nodes should look.
# The label and hover text will be the asset type's name.
Node.extend(AssetType, label=AssetType.asset_type, hover=AssetType.asset_type)

Node.extend(AssetTypeSpecific, label=AssetTypeSpecific.asset_type_specific)


# --- 7. Display the Graph ---
# Call the visualize method on the graph object we have built and configured.
# This will generate and display the interactive graph visualization.
# Note: This command is typically run in an environment like a Jupyter notebook.
clientAssetOwnershipGraph.visualize().display(inline=True)


## Create a Graph of the assets owned by the client. 
### The client ID is included as part of each specific asset.  

In [ ]:
# Assume 'model' is an existing context manager from the RelationalAI SDK.
# Assume 'Asset_Ownership' is a pre-existing base relation with columns:
# client_id, asset_type, asset_type_specific

# --- 1. Define Node Types ---
# A type for each unique client.
Client = model.Type("Client")

# A type for each unique asset type (e.g., "ETF", "Equity", "Bond").
AssetType = model.Type("AssetType")

# A type for a client's specific holding of an asset.
ClientSpecificAsset = model.Type("ClientSpecificAsset")
# The composite key is defined by declaring the fields that comprise it.
ClientSpecificAsset.asset_type_specific.declare()
ClientSpecificAsset.client_id.declare()
# Define a new property to hold the computed label for visualization.
ClientSpecificAsset.label.declare()


# --- 2. Populate Node Types from Source Data (with Cleaning) ---
# Import 'graphs' and the 'strings' module for data cleaning.
from relationalai.std import graphs, strings

# For each row in Asset_Ownership, ensure a Client entity exists.
with model.rule():
    t = Asset_Ownership()
    cleaned_id = strings.lowercase(strings.strip(t.client_id))
    t.set(client_id = Client.add(client_id=cleaned_id))

# For each row in Asset_Ownership, ensure an AssetType entity exists.
with model.rule():
    t = Asset_Ownership()
    cleaned_asset_type = strings.lowercase(strings.strip(t.asset_type))
    t.set(asset_type = AssetType.add(asset_type=cleaned_asset_type))

# For each row, ensure a ClientSpecificAsset entity exists.
with model.rule():
    t = Asset_Ownership()
    cleaned_asset_specific = strings.lowercase(strings.strip(t.asset_type_specific))
    cleaned_client_id = strings.lowercase(strings.strip(t.client_id))
    # Add the node with its composite key
    ClientSpecificAsset.add(asset_type_specific=cleaned_asset_specific, client_id=cleaned_client_id)

# NEW RULE: Compute and set the label property for each specific asset.
# This happens "in context", which resolves the error.
with model.rule():
    asset = ClientSpecificAsset()
    # Concatenate the asset name and client ID to create the label text.
    computed_label = strings.concat(
        asset.asset_type_specific, " (", asset.client_id, ")"
    )
    # Set the 'label' property on the entity.
    asset.set(label = computed_label)


# --- 3. Define and Build the Graph ---
# Create a graph from the model. This will be our single source of truth.
clientAssetOwnershipGraph = graphs.Graph(model, "client_asset_ownership_graph")


# --- 4. Add Nodes to the Graph ---
# Add all defined entities as nodes in the graph.
with model.rule():
    client = Client()
    client.set(clientAssetOwnershipGraph.Node)
with model.rule():
    assetType = AssetType()
    assetType.set(clientAssetOwnershipGraph.Node)
with model.rule():
    specificAsset = ClientSpecificAsset()
    specificAsset.set(clientAssetOwnershipGraph.Node)


# --- 5. Add Edges to the Graph (with Cleaning) ---
# Edge: Client -> AssetType
with model.rule():
    ownership = Asset_Ownership()
    from_node = Client(client_id=strings.lowercase(strings.strip(ownership.client_id)))
    to_node = AssetType(asset_type=strings.lowercase(strings.strip(ownership.asset_type)))
    clientAssetOwnershipGraph.Edge.add(from_=from_node, to=to_node)

# Edge: AssetType -> ClientSpecificAsset
with model.rule():
    ownership = Asset_Ownership()
    from_node = AssetType(asset_type=strings.lowercase(strings.strip(ownership.asset_type)))
    # Construct the 'to_node' using the composite key
    to_node = ClientSpecificAsset(
        asset_type_specific=strings.lowercase(strings.strip(ownership.asset_type_specific)),
        client_id=strings.lowercase(strings.strip(ownership.client_id))
    )
    clientAssetOwnershipGraph.Edge.add(from_=from_node, to=to_node)


# --- 6. Configure Visualization ---
Node = clientAssetOwnershipGraph.Node

# Configure Client and AssetType nodes.
Node.extend(Client, label=Client.client_id, hover=Client.client_id)
Node.extend(AssetType, label=AssetType.asset_type, hover=AssetType.asset_type)

# Configure the ClientSpecificAsset node label.
# We now refer to the pre-computed 'label' property.
Node.extend(ClientSpecificAsset, label=ClientSpecificAsset.label)


# --- 7. Display the Graph ---
# Call the visualize method on the graph object we have built and configured.
clientAssetOwnershipGraph.visualize().display(inline=True)


## Duplicate of above cell, but includes Jaccard Similarity.  

In [ ]:
# Assume 'model' is an existing context manager from the RelationalAI SDK.
# Assume 'Asset_Ownership' is a pre-existing base relation with columns:
# client_id, asset_type, asset_type_specific

# --- 1. Define Node Types ---
# A type for each unique client.
Client = model.Type("Client")

# A type for each unique asset type (e.g., "ETF", "Equity", "Bond").
AssetType = model.Type("AssetType")

# A type for a client's specific holding of an asset.
ClientSpecificAsset = model.Type("ClientSpecificAsset")
# The composite key is defined by declaring the fields that comprise it.
ClientSpecificAsset.asset_type_specific.declare()
ClientSpecificAsset.client_id.declare()
# Define a new property to hold the computed label for visualization.
ClientSpecificAsset.label.declare()


# --- 2. Populate Node Types from Source Data (with Cleaning) ---
# Import 'graphs' and the 'strings' module for data cleaning.
from relationalai.std import graphs, strings, alias

# For each row in Asset_Ownership, ensure a Client entity exists.
with model.rule():
    t = Asset_Ownership()
    cleaned_id = strings.lowercase(strings.strip(t.client_id))
    t.set(client_id = Client.add(client_id=cleaned_id))

# For each row in Asset_Ownership, ensure an AssetType entity exists.
with model.rule():
    t = Asset_Ownership()
    cleaned_asset_type = strings.lowercase(strings.strip(t.asset_type))
    t.set(asset_type = AssetType.add(asset_type=cleaned_asset_type))

# For each row, ensure a ClientSpecificAsset entity exists.
with model.rule():
    t = Asset_Ownership()
    cleaned_asset_specific = strings.lowercase(strings.strip(t.asset_type_specific))
    cleaned_client_id = strings.lowercase(strings.strip(t.client_id))
    # Add the node with its composite key
    ClientSpecificAsset.add(asset_type_specific=cleaned_asset_specific, client_id=cleaned_client_id)

# NEW RULE: Compute and set the label property for each specific asset.
# This happens "in context", which resolves the error.
with model.rule():
    asset = ClientSpecificAsset()
    # Concatenate the asset name and client ID to create the label text.
    computed_label = strings.concat(
        asset.asset_type_specific, " (", asset.client_id, ")"
    )
    # Set the 'label' property on the entity.
    asset.set(label = computed_label)


# --- 3. Define and Build the Graph ---
# Create a graph from the model. This will be our single source of truth.
clientAssetOwnershipGraph = graphs.Graph(model, "client_asset_ownership_graph")


# --- 4. Add Nodes to the Graph ---
# Add all defined entities as nodes in the graph.
with model.rule():
    client = Client()
    client.set(clientAssetOwnershipGraph.Node)
with model.rule():
    assetType = AssetType()
    assetType.set(clientAssetOwnershipGraph.Node)
with model.rule():
    specificAsset = ClientSpecificAsset()
    specificAsset.set(clientAssetOwnershipGraph.Node)


# --- 5. Add Edges to the Graph (with Cleaning) ---
# Edge: Client -> AssetType
with model.rule():
    ownership = Asset_Ownership()
    from_node = Client(client_id=strings.lowercase(strings.strip(ownership.client_id)))
    to_node = AssetType(asset_type=strings.lowercase(strings.strip(ownership.asset_type)))
    clientAssetOwnershipGraph.Edge.add(from_=from_node, to=to_node)

# Edge: AssetType -> ClientSpecificAsset
with model.rule():
    ownership = Asset_Ownership()
    from_node = AssetType(asset_type=strings.lowercase(strings.strip(ownership.asset_type)))
    # Construct the 'to_node' using the composite key
    to_node = ClientSpecificAsset(
        asset_type_specific=strings.lowercase(strings.strip(ownership.asset_type_specific)),
        client_id=strings.lowercase(strings.strip(ownership.client_id))
    )
    clientAssetOwnershipGraph.Edge.add(from_=from_node, to=to_node)


# --- 6. Configure Visualization ---
Node = clientAssetOwnershipGraph.Node

# Configure Client and AssetType nodes.
Node.extend(Client, label=Client.client_id, hover=Client.client_id)
Node.extend(AssetType, label=AssetType.asset_type, hover=AssetType.asset_type)

# Configure the ClientSpecificAsset node label.
# We now refer to the pre-computed 'label' property.
Node.extend(ClientSpecificAsset, label=ClientSpecificAsset.label)


# --- 7. Display the Graph ---
# Call the visualize method on the graph object we have built and configured.
clientAssetOwnershipGraph.visualize().display(inline=True)


# --- 8. Jaccard Similarity for Client Portfolios ---

# To calculate similarity based on specific assets, we need a graph
# that connects clients directly to the assets they own.

# Define a simple type for the ticker symbols to act as shared nodes.
TickerSymbol = model.Type("TickerSymbol")
TickerSymbol.ticker_symbol.declare()

# Populate the TickerSymbol type from the source data.
with model.rule():
    t = Asset_Ownership()
    cleaned_ticker = strings.lowercase(strings.strip(t.asset_type_specific))
    TickerSymbol.add(ticker_symbol=cleaned_ticker)

# Create a new, separate graph for the similarity calculation.
similarityGraph = graphs.Graph(model, "client_similarity_graph")

# Add Client and TickerSymbol entities as nodes to the new graph.
with model.rule():
    client = Client()
    similarityGraph.Node.add(client)
with model.rule():
    ticker = TickerSymbol()
    similarityGraph.Node.add(ticker)

# Add edges connecting each Client to the TickerSymbols they own.
with model.rule():
    ownership = Asset_Ownership()
    client_node = Client(client_id=strings.lowercase(strings.strip(ownership.client_id)))
    ticker_node = TickerSymbol(ticker_symbol=strings.lowercase(strings.strip(ownership.asset_type_specific)))
    similarityGraph.Edge.add(from_=client_node, to=ticker_node)


# Query: Who has a similar portfolio to 'client_a'?
with model.query() as select:
    # Define the client we are comparing against.
    # Note: the client_id must match the cleaned version used in the model.
    target_client = Client(client_id='client_g')

    # Iterate through all other clients.
    other_client = Client()
    other_client.client_id != 'client_g'

    # Compute Jaccard similarity based on shared specific assets in the similarityGraph.
    score = similarityGraph.compute.jaccard_similarity(other_client, target_client)

    # Select the client name and their similarity score.
    response = select(
        other_client.client_id,
        alias(score, "similarity_to_client_g")
    )
print('Jaccard Similarity')
response
# The 'response' object is now defined. To get the results, you would execute it,
# for example: results = conn.execute(response)


## Compute Jaccard Similarity for Client G. 

In [ ]:
# Query: Who has a similar portfolio to 'client_a'?
with model.query() as select:
    # Define the client we are comparing against.
    # Note: the client_id must match the cleaned version used in the model.
    target_client = Client(client_id='client_g')

    # Iterate through all other clients.
    other_client = Client()
    other_client.client_id != 'client_g'

    # Compute Jaccard similarity based on shared specific assets in the similarityGraph.
    score = similarityGraph.compute.jaccard_similarity(other_client, target_client)

    # Select the client name and their similarity score.
    response = select(
        other_client.client_id,
        alias(score, "similarity_to_client_g")
    )
response

## According to the Jaccard Similarity Scores, Client G & Client J have much in common.  
### Let's visualize the graphs for Client G & Client J to learn more about their portfolio holding.  

## Visualize the Portfolio of Client G.  

In [ ]:
# --- 9. Visualize a Single Client's Portfolio ---

# This code assumes the Types (Client, AssetType, ClientSpecificAsset) and
# helper modules (graphs, strings) from the previous cell are available.

# Create a new, separate graph for this specific visualization.
singleClientGraph = graphs.Graph(model, "single_client_portfolio_graph")

# Define the client we want to visualize.
# This must match the cleaned version (lowercase, no whitespace).
target_client_id = 'client_g'

# Rule to add nodes and edges related ONLY to the target client.
with model.rule():
    ownership = Asset_Ownership()
    # Apply a relational filter within the rule's context.
    # This ensures the rest of the rule only executes for matching rows.
    strings.lowercase(strings.strip(ownership.client_id)) == target_client_id

    # Define all the nodes based on this filtered data.
    client_node = Client(client_id=target_client_id)
    asset_type_node = AssetType(asset_type=strings.lowercase(strings.strip(ownership.asset_type)))
    specific_asset_node = ClientSpecificAsset(
        asset_type_specific=strings.lowercase(strings.strip(ownership.asset_type_specific)),
        client_id=target_client_id
    )

    # Add all relevant nodes to the new graph.
    singleClientGraph.Node.add(client_node)
    singleClientGraph.Node.add(asset_type_node)
    singleClientGraph.Node.add(specific_asset_node)

    # Add the edges between these nodes.
    singleClientGraph.Edge.add(from_=client_node, to=asset_type_node)
    singleClientGraph.Edge.add(from_=asset_type_node, to=specific_asset_node)

# Configure the visualization for the new, filtered graph.
SingleClientNode = singleClientGraph.Node
SingleClientNode.extend(Client, label=Client.client_id)
SingleClientNode.extend(AssetType, label=AssetType.asset_type)
# Use the pre-computed label from the main model for the specific asset nodes.
SingleClientNode.extend(ClientSpecificAsset, label=ClientSpecificAsset.label)

# Display the new graph containing only the target client's portfolio.
singleClientGraph.visualize().display(inline=True)


## Visualize the Portfolio Holdings for Client J

In [ ]:
# --- 9. Visualize a Single Client's Portfolio ---

# This code assumes the Types (Client, AssetType, ClientSpecificAsset) and
# helper modules (graphs, strings) from the previous cell are available.

# Create a new, separate graph for this specific visualization.
singleClientGraph = graphs.Graph(model, "single_client_portfolio_graph")

# Define the client we want to visualize.
# This must match the cleaned version (lowercase, no whitespace).
target_client_id = 'client_j'

# Rule to add nodes and edges related ONLY to the target client.
with model.rule():
    ownership = Asset_Ownership()
    # Apply a relational filter within the rule's context.
    # This ensures the rest of the rule only executes for matching rows.
    strings.lowercase(strings.strip(ownership.client_id)) == target_client_id

    # Define all the nodes based on this filtered data.
    client_node = Client(client_id=target_client_id)
    asset_type_node = AssetType(asset_type=strings.lowercase(strings.strip(ownership.asset_type)))
    specific_asset_node = ClientSpecificAsset(
        asset_type_specific=strings.lowercase(strings.strip(ownership.asset_type_specific)),
        client_id=target_client_id
    )

    # Add all relevant nodes to the new graph.
    singleClientGraph.Node.add(client_node)
    singleClientGraph.Node.add(asset_type_node)
    singleClientGraph.Node.add(specific_asset_node)

    # Add the edges between these nodes.
    singleClientGraph.Edge.add(from_=client_node, to=asset_type_node)
    singleClientGraph.Edge.add(from_=asset_type_node, to=specific_asset_node)

# Configure the visualization for the new, filtered graph.
SingleClientNode = singleClientGraph.Node
SingleClientNode.extend(Client, label=Client.client_id)
SingleClientNode.extend(AssetType, label=AssetType.asset_type)
# Use the pre-computed label from the main model for the specific asset nodes.
SingleClientNode.extend(ClientSpecificAsset, label=ClientSpecificAsset.label)

# Display the new graph containing only the target client's portfolio.
singleClientGraph.visualize().display(inline=True)


In [ ]:
# Assume 'model' is an existing context manager from the RelationalAI SDK.
# Assume 'Asset_Ownership' is a pre-existing base relation with columns:
# client_id, asset_type, asset_type_specific, quantity, purchase_price, current_price, purchase_date

# --- 1. Define Node Types ---
# A type for each unique client.
Client = model.Type("Client")

# A type for each unique asset type (e.g., "ETF", "Equity", "Bond").
AssetType = model.Type("AssetType")

# A type for a client's specific holding of an asset.
# Based on the assumption of one transaction per asset, the key is simplified.
ClientSpecificAsset = model.Type("ClientSpecificAsset")
ClientSpecificAsset.asset_type_specific.declare()
ClientSpecificAsset.client_id.declare()
# Define properties to hold the label and financial data.
ClientSpecificAsset.label.declare()
ClientSpecificAsset.quantity.declare()
ClientSpecificAsset.purchase_price.declare()
ClientSpecificAsset.current_price.declare()
ClientSpecificAsset.purchase_date.declare()


# --- 2. Populate Node Types and Properties from Source Data ---
# Import necessary modules and functions.
from relationalai.std import graphs, strings, alias

# For each row in Asset_Ownership, ensure a Client entity exists.
with model.rule():
    t = Asset_Ownership()
    cleaned_id = strings.lowercase(strings.strip(t.client_id))
    t.set(client_id = Client.add(client_id=cleaned_id))

# For each row in Asset_Ownership, ensure an AssetType entity exists.
with model.rule():
    t = Asset_Ownership()
    cleaned_asset_type = strings.lowercase(strings.strip(t.asset_type))
    t.set(asset_type = AssetType.add(asset_type=cleaned_asset_type))

# For each row, ensure a ClientSpecificAsset entity exists.
with model.rule():
    t = Asset_Ownership()
    cleaned_asset_specific = strings.lowercase(strings.strip(t.asset_type_specific))
    cleaned_client_id = strings.lowercase(strings.strip(t.client_id))
    # Add the node with its simplified composite key.
    ClientSpecificAsset.add(
        asset_type_specific=cleaned_asset_specific,
        client_id=cleaned_client_id
    )

# NEW RULE: Compute and set the properties for each specific asset.
with model.rule():
    # We join the entity with the source data to get all the properties.
    asset = ClientSpecificAsset()
    ownership = Asset_Ownership()
    # Simplified join condition.
    asset.asset_type_specific == strings.lowercase(strings.strip(ownership.asset_type_specific))
    asset.client_id == strings.lowercase(strings.strip(ownership.client_id))

    # Create a single, detailed label by concatenating all desired information.
    # The `strings.concat` function will handle the conversion of numbers and dates.
    computed_label = strings.concat(
        asset.asset_type_specific, " (", asset.client_id, ")\n",
        "Qty: ", ownership.quantity, ", Price: ", ownership.purchase_price, "\n",
        "Date: ", ownership.purchase_date
    )
    # Set all the properties on the entity.
    asset.set(
        label = computed_label,
        quantity = ownership.quantity,
        purchase_price = ownership.purchase_price,
        current_price = ownership.current_price,
        purchase_date = ownership.purchase_date
    )


# --- 3. Define and Build the Graph ---
# Create a graph from the model. This will be our single source of truth.
clientAssetOwnershipGraph = graphs.Graph(model, "client_asset_ownership_graph")


# --- 4. Add Nodes to the Graph ---
# Add all defined entities as nodes in the graph.
with model.rule():
    client = Client()
    client.set(clientAssetOwnershipGraph.Node)
with model.rule():
    assetType = AssetType()
    assetType.set(clientAssetOwnershipGraph.Node)
with model.rule():
    specificAsset = ClientSpecificAsset()
    specificAsset.set(clientAssetOwnershipGraph.Node)


# --- 5. Add Edges to the Graph (with Cleaning) ---
# Edge: Client -> AssetType
with model.rule():
    ownership = Asset_Ownership()
    from_node = Client(client_id=strings.lowercase(strings.strip(ownership.client_id)))
    to_node = AssetType(asset_type=strings.lowercase(strings.strip(ownership.asset_type)))
    clientAssetOwnershipGraph.Edge.add(from_=from_node, to=to_node)

# Edge: AssetType -> ClientSpecificAsset
with model.rule():
    ownership = Asset_Ownership()
    from_node = AssetType(asset_type=strings.lowercase(strings.strip(ownership.asset_type)))
    # Construct the 'to_node' using the simplified composite key.
    to_node = ClientSpecificAsset(
        asset_type_specific=strings.lowercase(strings.strip(ownership.asset_type_specific)),
        client_id=strings.lowercase(strings.strip(ownership.client_id))
    )
    clientAssetOwnershipGraph.Edge.add(from_=from_node, to=to_node)


# --- 6. Configure Visualization ---
Node = clientAssetOwnershipGraph.Node

# Configure Client and AssetType nodes.
Node.extend(Client, label=Client.client_id, hover=Client.client_id)
Node.extend(AssetType, label=AssetType.asset_type, hover=AssetType.asset_type)

# Configure the ClientSpecificAsset node label.
# We now refer to the pre-computed, detailed label property.
Node.extend(ClientSpecificAsset, label=ClientSpecificAsset.label)


# --- 7. Display the Graph ---
# Call the visualize method on the graph object we have built and configured.
clientAssetOwnershipGraph.visualize().display(inline=True)


# --- 8. Jaccard Similarity for Client Portfolios ---

# To calculate similarity based on specific assets, we need a graph
# that connects clients directly to the assets they own.

# Define a simple type for the ticker symbols to act as shared nodes.
TickerSymbol = model.Type("TickerSymbol")
TickerSymbol.ticker_symbol.declare()

# Populate the TickerSymbol type from the source data.
with model.rule():
    t = Asset_Ownership()
    cleaned_ticker = strings.lowercase(strings.strip(t.asset_type_specific))
    TickerSymbol.add(ticker_symbol=cleaned_ticker)

# Create a new, separate graph for the similarity calculation.
similarityGraph = graphs.Graph(model, "client_similarity_graph")

# Add Client and TickerSymbol entities as nodes to the new graph.
with model.rule():
    client = Client()
    similarityGraph.Node.add(client)
with model.rule():
    ticker = TickerSymbol()
    similarityGraph.Node.add(ticker)

# Add edges connecting each Client to the TickerSymbols they own.
with model.rule():
    ownership = Asset_Ownership()
    client_node = Client(client_id=strings.lowercase(strings.strip(ownership.client_id)))
    ticker_node = TickerSymbol(ticker_symbol=strings.lowercase(strings.strip(ownership.asset_type_specific)))
    similarityGraph.Edge.add(from_=client_node, to=ticker_node)


# Query: Who has a similar portfolio to 'client_a'?
with model.query() as select:
    # Define the client we are comparing against.
    # Note: the client_id must match the cleaned version used in the model.
    target_client = Client(client_id='client_a')

    # Iterate through all other clients.
    other_client = Client()
    other_client.client_id != 'client_a'

    # Compute Jaccard similarity based on shared specific assets in the similarityGraph.
    score = similarityGraph.compute.jaccard_similarity(other_client, target_client)

    # Select the client name and their similarity score.
    response = select(
        other_client.client_id,
        alias(score, "similarity_to_client_a")
    )

# The 'response' object is now defined. To get the results, you would execute it,
# for example: results = conn.execute(response)


# 🚀 What's Next? From Insight to Action 💡
What we've shown you today is just the beginning 🌱. The Client Wealth Map is a living foundation for deeper, more powerful analysis.

## 🔎 Unmask Influence with Centrality Measures
- Identify Key Connectors 🔗: Discover which assets or goals are most central to your clients' portfolios.
- Understand Client Needs 🧠: Gain a deeper understanding of your clients' core financial drivers and motivations.

## 🔑 Unlock Hidden Segments with Community Detection
- Hyper-Personalization 🎯: Automatically segment clients into nuanced groups based on their entire financial DNA, not just simple demographics.
- Spot Emerging Trends 📈: Discover hidden communities of clients with shared interests, like a growing interest in ESG investing.

## 💰 Drive Business Growth
- Enhanced Product Development ⚙️: Use community insights to build new, tailored products—like an ESG-focused investment fund—for a well-defined, organic client base.
- Targeted Engagement 📣: Create specific educational content and advisory services that resonate deeply with each discovered community.

## ⚡ The Power of the Modern Data Stack ⚡
## Snowflake ❄️ + RelationalAI 🧠
#### By combining the unlimited scale and security 🛡️ of the Snowflake Data Cloud with the declarative power 💪 of the Relational AI coprocessor, building a deep, actionable understanding of your clients has never been more straightforward. ✅

Thank you for watching! 👋